# Capstone Project - The Batttle of the Neighborhoods Final

## IBM Data Science Professional Certificate Applied Capstone Project

## Business Problem

Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

#### The problem that is to be solved by our application is where is the best neighbhorhood in Toronto for our clients or stakeholders to move into based on what amenties are located nearby.

#### Toronto is considered one of the most popular cities in Canada. Many people make the decision to move there every year to different neighbhorhoods for a variety of reasons. A number of the existing popluation even choose to move to different places within the city for various reasons. By utilizing FourSquare location data, we will be able to segment and cluster neighborhoods together and collect data on what amenties are in each neighborhood. By collecting type and frequency of venues nearby, the client or stakeholder will be able to determine where they think is the best place for them to move. As apart of a larger question to be solved, many choose their residence based on what is nearby making this application useful to those looking to move to a neighborhood close to their favorite businesses or places.

## Data

Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

#### In order to provide necessary information to our clients or to the stakeholders, Foursquare API to collect data such as type and frequency for venues in each neighborhood. By segmenting and clustering, we can find the top and most popular venues in each neighborhood in Toronto. This will give the end user an idea of what kind of ammenties are and where there are located.

#### Data to be used in this project will be as follows:

#### List of postal codes of Canada Wikipedia for neighborhoods: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

#### Foursquare API data for amenties, shops, and resturants in each neighborhood of Toronto

### Import Libraries

In [1]:
import numpy as np
import pandas as pd 
import json

!pip install wikipedia
import wikipedia as wp

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes

from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print("Import Complete")

  Stored in directory: /home/dsxuser/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                          

### Create Table of Toronto Neighborhoods with Geographical coordinates.

We need to know where exactly each neighbhorhood is located in Toronto. We first create a table containing neighborhood name and it's geographical location using longitude and latitude.

In [32]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")

df = pd.read_html(html)[0]
df = df[df.Community!= "Not assigned"]

df = df.groupby(['Postal Code', 'Community'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Community']

column_names = ['Postal Code','Community', 'Neighborhood'] 
df.columns = column_names

df.head()

,Postal Code,Community,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [33]:
import pandas as pd
import io
import requests

url="https://cocl.us/Geospatial_data"

s=requests.get(url).content

c=pd.read_csv(io.StringIO(s.decode('utf-8')))

dfc = df.join(c.set_index('Postal Code'), on='Postal Code')

dfc.head()

,Postal Code,Community,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [34]:
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('Coordinates of Toronto are {}, {}.'.format(latitude, longitude))

Coordinates of Toronto are 43.6534817, -79.3839347.


### Create Map of Toronto

To get a more visual understanding or where the neighborhoods are located in Toronto, we create a map using the Folium Map package.

In [35]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, district, neighborhood in zip(dfc['Latitude'], dfc['Longitude'], dfc['Community'], dfc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.2,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### FourSquare API

By using FourSquare API, we will be able to search and collect popular venues in Toronto. We will see what type, location, and frequency of specific venues located in the different neighborhoods. By using location data, we will be able to determine which neighborhood best suits the client or stakeholders needs and wants.

In [36]:
CLIENT_ID = '0LEACYSZ1UDN2OT3EADC5KUX4KYXBCUBVR0CC44RIB0SAD0N' 
CLIENT_SECRET = 'HTYT3TIWRTL2YAZWEVJVFFS4RT3RTSKBPIIOK0AWVN2RAQV3'

VERSION = '20180604'
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius = 500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

Your credentails:
CLIENT_ID: 0LEACYSZ1UDN2OT3EADC5KUX4KYXBCUBVR0CC44RIB0SAD0N
CLIENT_SECRET:HTYT3TIWRTL2YAZWEVJVFFS4RT3RTSKBPIIOK0AWVN2RAQV3


'https://api.foursquare.com/v2/venues/search?client_id=0LEACYSZ1UDN2OT3EADC5KUX4KYXBCUBVR0CC44RIB0SAD0N&client_secret=HTYT3TIWRTL2YAZWEVJVFFS4RT3RTSKBPIIOK0AWVN2RAQV3&ll=43.6534817,-79.3839347&v=20180604&radius=500&limit=100'

In [37]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [38]:
results = requests.get(url).json()
results

venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",False,4c093ee0340720a153728493,NaN,CA,NaN,Canada,NaN,184,[Canada],"[{'label': 'display', 'lat': 43.65182710471462...",43.651827,-79.383949,NaN,NaN,NaN,City Hall Council Chambers,v-1600563000
1,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",False,4ad4c05ef964a5208ff620e3,100 Queen St. W.,CA,Toronto,Canada,at Bay St.,38,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...","[{'label': 'display', 'lat': 43.65313989695342...",43.653140,-79.383967,NaN,M5H 2N2,ON,Toronto City Hall,v-1600563000
2,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",False,5f5e5ad065f5f93e83216382,258 Queens Quay W,CA,Toronto,Canada,NaN,1573,"[258 Queens Quay W, Toronto ON M5J 1B5, Canada]","[{'label': 'display', 'lat': 43.639397, 'lng':...",43.639397,-79.385605,NaN,M5J 1B5,ON,Bubble Baby,v-1600563000
3,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",False,5b193c42598e64002ca79b96,100 Queen St W,CA,Toronto,Canada,NaN,3,"[100 Queen St W, Toronto ON M5H 2N2, Canada]","[{'label': 'display', 'lat': 43.653454, 'lng':...",43.653454,-79.383952,NaN,M5H 2N2,ON,City of Toronto Civic Innovation Office,v-1600563000
4,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,50885719498ea7b5aab3a74c,483 Bay St,CA,Toronto,Canada,NaN,130,"[483 Bay St, Toronto ON M5G 2C9, Canada]","[{'label': 'display', 'lat': 43.653436, 'lng':...",43.653436,-79.382314,NaN,M5G 2C9,ON,GoodLife Fitness Toronto Bell Trinity Centre,v-1600563000


In [39]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']

dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,City Hall Council Chambers,City Hall,NaN,CA,NaN,Canada,NaN,184,[Canada],"[{'label': 'display', 'lat': 43.65182710471462...",43.651827,-79.383949,NaN,NaN,NaN,4c093ee0340720a153728493
1,Toronto City Hall,City Hall,100 Queen St. W.,CA,Toronto,Canada,at Bay St.,38,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...","[{'label': 'display', 'lat': 43.65313989695342...",43.653140,-79.383967,NaN,M5H 2N2,ON,4ad4c05ef964a5208ff620e3
2,Bubble Baby,Bubble Tea Shop,258 Queens Quay W,CA,Toronto,Canada,NaN,1573,"[258 Queens Quay W, Toronto ON M5J 1B5, Canada]","[{'label': 'display', 'lat': 43.639397, 'lng':...",43.639397,-79.385605,NaN,M5J 1B5,ON,5f5e5ad065f5f93e83216382
3,City of Toronto Civic Innovation Office,City Hall,100 Queen St W,CA,Toronto,Canada,NaN,3,"[100 Queen St W, Toronto ON M5H 2N2, Canada]","[{'label': 'display', 'lat': 43.653454, 'lng':...",43.653454,-79.383952,NaN,M5H 2N2,ON,5b193c42598e64002ca79b96
4,GoodLife Fitness Toronto Bell Trinity Centre,Gym,483 Bay St,CA,Toronto,Canada,NaN,130,"[483 Bay St, Toronto ON M5G 2C9, Canada]","[{'label': 'display', 'lat': 43.653436, 'lng':...",43.653436,-79.382314,NaN,M5G 2C9,ON,50885719498ea7b5aab3a74c


In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
toronto_venues = getNearbyVenues(names=dfc['Neighborhood'],
                                   latitudes=dfc['Latitude'],
                                   longitudes=dfc['Longitude']
                                  )

toronto_venues.head()

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [42]:
print('{} venues were returned by Foursquare.'.format(toronto_venues.shape[0]))
toronto_venues.head()

2145 venues were returned by Foursquare.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [43]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,55,55,55,55,55,55
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16


In [44]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


This is where we use Foursquare API to start to look at what type of venues are there, frequency, and location in the neighborhoods. The information given by FourSquare API will give our clients or stakeholders the exact information they are looking for in order to make their decision on which neighborhood to move too.

In [45]:
#### One Hot Encoding

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])

manhattan_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [46]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,

We will use the top 10 most popular venues in order to get a better picture of what each of the neighborhoods are like. By using 10 different venues we will see various resturants, stores, and other businesses in the area that people might want to visit regularly. 

In [47]:
num_top_venues = 11

for hood in toronto_grouped['Neighborhood']:
    
    print("----"+hood+"----")
    
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
   
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                              venue  freq
0         Latin American Restaurant   0.2
1                    Breakfast Spot   0.2
2                            Lounge   0.2
3                      Skating Rink   0.2
4                    Clothing Store   0.2
5                 Accessories Store   0.0
6               Monument / Landmark   0.0
7   Molecular Gastronomy Restaurant   0.0
8        Modern European Restaurant   0.0
9                 Mobile Phone Shop   0.0
10               Miscellaneous Shop   0.0


----Alderwood, Long Branch----
                       venue  freq
0                Pizza Place  0.29
1                        Gym  0.14
2                        Pub  0.14
3                Coffee Shop  0.14
4                   Pharmacy  0.14
5             Sandwich Place  0.14
6   Mediterranean Restaurant  0.00
7                Men's Store  0.00
8             Medical Center  0.00
9              Metro Station  0.00
10        Mexican Restaurant  0.00


----Bathurst Manor, Wil

                  venue  freq
0           Coffee Shop  0.12
1            Restaurant  0.07
2                 Hotel  0.06
3                  Café  0.06
4   American Restaurant  0.04
5                   Gym  0.04
6   Japanese Restaurant  0.03
7    Seafood Restaurant  0.03
8             Gastropub  0.03
9        Breakfast Spot  0.02
10   Italian Restaurant  0.02


----Davisville----
                 venue  freq
0          Pizza Place  0.09
1       Sandwich Place  0.09
2         Dessert Shop  0.09
3                 Café  0.06
4     Sushi Restaurant  0.06
5                  Gym  0.06
6          Coffee Shop  0.06
7   Italian Restaurant  0.06
8                 Park  0.03
9      Thai Restaurant  0.03
10             Brewery  0.03


----Davisville North----
                   venue  freq
0         Breakfast Spot  0.11
1                  Hotel  0.11
2       Department Store  0.11
3           Dance Studio  0.11
4         Sandwich Place  0.11
5   Gym / Fitness Center  0.11
6                Dog Run  0

                              venue  freq
0                             Field  0.25
1                      Tennis Court  0.25
2                      Hockey Arena  0.25
3                             Trail  0.25
4                Mexican Restaurant  0.00
5   Molecular Gastronomy Restaurant  0.00
6        Modern European Restaurant  0.00
7                 Mobile Phone Shop  0.00
8                Miscellaneous Shop  0.00
9         Middle Eastern Restaurant  0.00
10                    Metro Station  0.00


----India Bazaar, The Beaches West----
                   venue  freq
0                   Park  0.11
1      Fish & Chips Shop  0.05
2     Italian Restaurant  0.05
3              Pet Store  0.05
4                    Pub  0.05
5            Coffee Shop  0.05
6             Restaurant  0.05
7          Movie Theater  0.05
8         Sandwich Place  0.05
9          Burrito Place  0.05
10  Fast Food Restaurant  0.05


----Kennedy Park, Ionview, East Birchmount Park----
                             

                        venue  freq
0              Massage Studio  0.14
1                 Coffee Shop  0.14
2        Caribbean Restaurant  0.14
3                         Bar  0.14
4      Furniture / Home Store  0.14
5          Miscellaneous Shop  0.14
6               Metro Station  0.14
7          Mexican Restaurant  0.00
8   Middle Eastern Restaurant  0.00
9               Movie Theater  0.00
10          Mobile Phone Shop  0.00


----Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East----
                              venue  freq
0        Construction & Landscaping   0.5
1                    Baseball Field   0.5
2                 Accessories Store   0.0
3                Miscellaneous Shop   0.0
4               Moroccan Restaurant   0.0
5               Monument / Landmark   0.0
6   Molecular Gastronomy Restaurant   0.0
7        Modern European Restaurant   0.0
8                 Mobile Phone Shop   0.0
9  

                  venue  freq
0           Coffee Shop  0.13
1           Pizza Place  0.07
2            Sports Bar  0.07
3                  Bank  0.07
4            Bagel Shop  0.07
5           Supermarket  0.07
6            Restaurant  0.07
7    Light Rail Station  0.07
8   Fried Chicken Joint  0.07
9                   Pub  0.07
10  American Restaurant  0.07


----The Annex, North Midtown, Yorkville----
                        venue  freq
0              Sandwich Place  0.16
1                        Café  0.16
2                 Coffee Shop  0.11
3                         Pub  0.05
4                  Donut Shop  0.05
5                   BBQ Joint  0.05
6                Liquor Store  0.05
7                Burger Joint  0.05
8                 Pizza Place  0.05
9   Middle Eastern Restaurant  0.05
10                       Park  0.05


----The Beaches----
                              venue  freq
0                 Health Food Store  0.25
1                             Trail  0.25
2             

In [48]:
def return_most_common_venues(row, num_top_venues):
    
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Clothing Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Coffee Shop,Pub,Pharmacy,Gym,Gourmet Shop,Discount Store,Deli / Bodega,Department Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pharmacy,Restaurant,Supermarket,Deli / Bodega,Sushi Restaurant,Mobile Phone Shop,Fried Chicken Joint,Ice Cream Shop
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Grocery Store,Thai Restaurant,Liquor Store,Comfort Food Restaurant,Juice Bar,Butcher,Café


By this point, we are starting to see the make up of neighborhoods around Toronto. As you can see in the table, FourSquare API can give very specific information about the businesses that can be found. By using that information we are able to cater our application to our clients and stakeholders. While many people may put the same importance on the first few venues, most common venues 4 - 10 can greatly differ in importance to different users. The more important venues show up on the list, the more likely a user is to find that neighborhood more attractive and willing to move there. 

## Methodology

In this project, we took a look at what type of venues, where they are located, and how often does a specific type of venue appear within the various neighborhoods in Toronto. 

First, our application took neighborhood data from a wikipedia page in order to find the exact location (longitude and latitude) of each of the different neighborhoods. After marking the locations of each neighborhood, we were able to visualize where the neighborhoods are located by producing a map from the folium package. 

Second, we used FourSquare API location data to find all the different types of venues located in the various neighborhoods around Toronto. This gave us an idea of what type and how many venues are located within a specific neighborhood. FourSquare returned **2145** venues around Toronto. This produced **274** unique categories that we can use to organize our data in. 

Third, we were able to take the FourSquare and location data and place them into the neighborhoods they are located in. From there we can cluster the neighborhoods together then create a list of the **10** most common venues found.

Finally, after looking through the list of neighborhoods and their venues, we can choose which neighborhoods hold the most interest for our users and compare them. By comparing the neighborhoods side by side, we can see their locations and what the **10** most common venues are next to each other. This will give our user a very visual display of what the neighborhoods are like and use that information to help them make a decision.

## Analysis

Next we will cluster the neighborhoods together so we can get a better picture of what venues are where and even which neighborhoods are close to each other. This will allow our user to analize the data and see not only what is in their neighborhood of choice, but also what may be nearby for them to travel too. After clustering the neighborhoods, we will be able to compare neighborhoods together to better see which neighborhoods suit our users best.

### Cluster Neighborhoods 

In [ ]:
kclusters = 2
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [ ]:
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)
toronto_merged = dfc

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged.dropna()
toronto_merged.head()

,Postal Code,Community,Neighborhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Hardware Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1.0,Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Intersection,Mexican Restaurant,Breakfast Spot,Electronics Store,Restaurant,Rental Car Location,Medical Center,Bank,Discount Store,Distribution Center
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Mexican Restaurant,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Caribbean Restaurant,Fried Chicken Joint,Bank,Gas Station,Athletics & Sports,Thai Restaurant,Bakery,Hakka Restaurant,Eastern European Restaurant,Dumpling Restaurant


Now we can map the different clusters and see which neighborhoods are located near each other. Although a user might decide to move to a neighborhood for specific venues, they also might choose that location due to the neighborhoods nearby and what venues are located there. If neighborhoods nearby contain venues they would be interested in, it would not be far to travel to visit those places. This might make certain neighborhoods more attractive not just based the venues on the neighborhood itself, but also what venues are located in nearby neighborhoods.

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
df1=toronto_merged.loc[toronto_merged['ClusterLabels'] ==1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
df1

,Community,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Hardware Store
1,Scarborough,1.0,Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,Scarborough,1.0,Intersection,Mexican Restaurant,Breakfast Spot,Electronics Store,Restaurant,Rental Car Location,Medical Center,Bank,Discount Store,Distribution Center
3,Scarborough,1.0,Coffee Shop,Korean Restaurant,Mexican Restaurant,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,Scarborough,1.0,Caribbean Restaurant,Fried Chicken Joint,Bank,Gas Station,Athletics & Sports,Thai Restaurant,Bakery,Hakka Restaurant,Eastern European Restaurant,Dumpling Restaurant
5,Scarborough,1.0,Playground,Smoke Shop,Jewelry Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
6,Scarborough,1.0,Train Station,Hobby Shop,Coffee Shop,Department Store,Escape Room,Ethiopian Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
7,Scarborough,1.0,Bakery,Bus Line,Park,Intersection,Ice Cream Shop,Metro Station,Bus Station,Soccer Field,Drugstore,Dumpling Restaurant
8,Scarborough,1.0,Motel,American Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop
9,Scarborough,1.0,College Stadium,General Entertainment,Skating Rink,Café,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


In [ ]:
clusters=pd.DataFrame({"Cluster1":df1["Community"],
                       
                      })

In [ ]:
clusters = clusters.replace(np.nan, '', regex=True)

In [ ]:
clusters

,Cluster1
0,Scarborough
1,Scarborough
2,Scarborough
3,Scarborough
4,Scarborough
5,Scarborough
6,Scarborough
7,Scarborough
8,Scarborough
9,Scarborough


In [ ]:
new_toronto=toronto_merged.set_index("Neighborhood",drop=True)
new_toronto.head(103)

,Postal Code,Community,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighborhood,,,,,,,,,,,,,,,
"Malvern, Rouge",M1B,Scarborough,43.806686,-79.194353,1.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Hardware Store
"Rouge Hill, Port Union, Highland Creek",M1C,Scarborough,43.784535,-79.160497,1.0,Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
"Guildwood, Morningside, West Hill",M1E,Scarborough,43.763573,-79.188711,1.0,Intersection,Mexican Restaurant,Breakfast Spot,Electronics Store,Restaurant,Rental Car Location,Medical Center,Bank,Discount Store,Distribution Center
Woburn,M1G,Scarborough,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Mexican Restaurant,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
Cedarbrae,M1H,Scarborough,43.773136,-79.239476,1.0,Caribbean Restaurant,Fried Chicken Joint,Bank,Gas Station,Athletics & Sports,Thai Restaurant,Bakery,Hakka Restaurant,Eastern European Restaurant,Dumpling Restaurant
Scarborough Village,M1J,Scarborough,43.744734,-79.239476,1.0,Playground,Smoke Shop,Jewelry Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
"Kennedy Park, Ionview, East Birchmount Park",M1K,Scarborough,43.727929,-79.262029,1.0,Train Station,Hobby Shop,Coffee Shop,Department Store,Escape Room,Ethiopian Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
"Golden Mile, Clairlea, Oakridge",M1L,Scarborough,43.711112,-79.284577,1.0,Bakery,Bus Line,Park,Intersection,Ice Cream Shop,Metro Station,Bus Station,Soccer Field,Drugstore,Dumpling Restaurant
"Cliffside, Cliffcrest, Scarborough Village West",M1M,Scarborough,43.716316,-79.239476,1.0,Motel,American Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop


### Compare Neighborhoods

Lets compare 2 neighborhoods within Toronto to see what are the most common venues in those areas. The neighborhoods we will test are Humber Summit in North York and Christie in Downtown Toronto. The 2 different neighborhoods we are testing are in different locations as a more northern area away that may be more family friendly, and another location that is located in the heart of the city which cater to different lifestyles in a busy environment.

In [ ]:
Nei1=input("Enter the Neighborhood: ")

In [ ]:
Nei2=input("Enter the Neighborhood: ")

In [ ]:
Venue_comparison=new_toronto.loc[[Nei1,Nei2]].T
Venue_comparison

## Results and Discussion

There are many neighborhoods and even more venues to consider. If our clients or stakeholders had to do this by themselves manually, it would take hours if not days to even find a little information that there are looking for. Based on our analysis, we have found the 10 most common venues in each neighborhood. Our clients or end users may have the same top 3 venues there are looking for which would make several neighborhoods appear the same which would not provide enough distinction between them. With **2145** venues in **274** unique categories with **103** different communities with some of those communities having **2 or more** neighborhoods in each one. There is a lot of area to cover in order to pick the right neighborhood for our clients.

We started with listing all the neighborhoods with their geographical coordinates (longitude/latitude). After that we pulled FourSquare location data on all the different type venues located in the neighborhoods in Toronto. With all the location data on venues, we were able to organize them, and sort out the most common venues in each neighborhood. We cluster the neighborhoods together in their respective communities and print out a table. 

The most informative part of this application is the section that the end user or client would use most. After looking at the list of neighborhoods and their venues, they can compare neighborhoods or communities together side by side to give them a more visual comparison. We pulled **Humber Summit** and **Christie** neighborhoods together for a side by side test comparison on their 10 most common venue. As mentioned before, the top 3 most common venues may be the same for a number of people, but what this application has is the 10 most common venues which may provide a more specific picture on the make up of the neighborhoods which would narrow down the search for our clients or end users.

Based on the comparison we ran with our test neighborhoods, **Humber Summit** located in **North York** seems more appealing in our test based on the number of *restaurants* and *stores* that *families* might like rather than **Christie** located in **Downtown Toronto** that seems to provide a more city lifestyle that appeals more to *younger individuals or individuals without families* as it has venues like *nightclubs* and *athletics & sports*.

## Conclusion

The purpose of this application is to help our clients, end users, and stakeholders compare different neighborhoods in Toronto based on their most commone venues. As people may have their top 3 most common venues the same, what separates our application from others is that it list out the **10** most common venues which may help users separate the different neighborhoods that may have the same top 3 venues. The list also includes neighborhoods and venues in different parts of Toronto that may offer a vastly different lifestyle from each other that appeal to a number of different people.

Whether the user is single, young, and looking to live it up in the city or part of a family looking for a quiet family friendly part of town just outside the hustle and bustle of the big city, our application is here to help people distinguish the different communities and neighborhoods that may cater to each and everyone's lifestyle. In depth information provide will help separate the neighborhoods apart and even show neighborhoods nearby that our users may also be interested in if they can live in a neighborhood nearby and travel just a short distance to even more venues making their choice of neighborhood. Addition factors can be added such as housing prices, school rating, and more to this base application. Those factors may not matter to everyone as income and family size if different. The types of venus, location, and frequency of them is something that would matter to more people due to the number of different businesses that cater to people of all backgrounds. So no matter who you are, where you come from, this application can you help you find a neighborhood for you in Toronto!